In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./docs/tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)                                                                                                            

In [3]:
len(document_list)

225

In [4]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model='gpt-4o-mini')

In [5]:
prompt_template = """
주어진 두 텍스트의 유사성을 비교하여 공통점과 차이점을 생성하여 답변합니다.

Chunk 1:
{chunk1}

Chunk 2:
{chunk2}

답변은 한글로 합니다.
답변은 서론-본론-결론으로 정리합니다.
유사도를 0~100으로 측정하여 결론에 포함시켜 답변합니다.
"""

In [6]:
chunk1 = document_list[51]
chunk2 = document_list[52]

input_data = {
    "chunk1": chunk1,
    "chunk2": chunk2
}

In [7]:
chunk1

Document(metadata={'source': './docs/tax.docx'}, page_content='③ 거주자의 부양가족 중 거주자(그 배우자를 포함한다)의 직계존속이 주거 형편에 따라 별거하고 있는 경우에는 제1항에도 불구하고 제50조에서 규정하는 생계를 같이 하는 사람으로 본다.\n\n④ 제50조, 제51조 및 제59조의2에 따른 공제대상 배우자, 공제대상 부양가족, 공제대상 장애인 또는 공제대상 경로우대자에 해당하는지 여부의 판정은 해당 과세기간의 과세기간 종료일 현재의 상황에 따른다. 다만, 과세기간 종료일 전에 사망한 사람 또는 장애가 치유된 사람에 대해서는 사망일 전날 또는 치유일 전날의 상황에 따른다.<개정 2014. 1. 1.>\n\n⑤ 제50조제1항제3호 및 제59조의2에 따라 적용대상 나이가 정해진 경우에는 제4항 본문에도 불구하고 해당 과세기간의 과세기간 중에 해당 나이에 해당되는 날이 있는 경우에 공제대상자로 본다.<개정 2014. 1. 1.>\n\n[전문개정 2009. 12. 31.]\n\n\n\n제54조(종합소득공제 등의 배제) ① 분리과세이자소득, 분리과세배당소득, 분리과세연금소득과 분리과세기타소득만이 있는 자에 대해서는 종합소득공제를 적용하지 아니한다. <개정 2013. 1. 1.>\n\n② 제70조제1항, 제70조의2제2항 또는 제74조에 따라 과세표준확정신고를 하여야 할 자가 제70조제4항제1호에 따른 서류를 제출하지 아니한 경우에는 기본공제 중 거주자 본인에 대한 분(分)과 제59조의4제9항에 따른 표준세액공제만을 공제한다. 다만, 과세표준확정신고 여부와 관계없이 그 서류를 나중에 제출한 경우에는 그러하지 아니하다.<개정 2013. 1. 1., 2014. 1. 1.>\n\n③ 제82조에 따른 수시부과 결정의 경우에는 기본공제 중 거주자 본인에 대한 분(分)만을 공제한다.\n\n[전문개정 2009. 12. 31.]\n\n[제목개정 2014. 1. 1.]\n\n\n\n제54조의2(공동사업에 대한 소득공제 등 특례) 제

In [8]:
chunk2

Document(metadata={'source': './docs/tax.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한다.<

In [9]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    input_variables=["chunk1", "chunk2"],
    template=prompt_template
)

chain = prompt | llm | StrOutputParser()
similarity_result = chain.invoke(input_data)

In [10]:
print(similarity_result)

### 서론
본 문서는 두 개의 텍스트 조각(Chunk 1과 Chunk 2)를 비교하여 공통점과 차이점을 분석하고, 이들의 유사성을 평가하는 데 목적을 두고 있다. 두 텍스트는 세법 관련 조항으로, 공제 대상, 과세 기준 및 세율에 대한 내용을 다루고 있다.

### 본론
**공통점:**
1. **세법 관련 내용**: 두 Chunk 모두 거주자의 소득세 및 관련 공제에 대한 내용을 포함하고 있으며, 세율과 과세표준에 대한 규정을 다루고 있다.
2. **제55조에 대한 언급**: 두 문서 모두 제55조를 인용하고 있으며, 종합소득에 대한 세율 계산 방법을 설명하고 있다. 
3. **개정 사항**: 두 문서 모두 과세 관련 조항에 대해 개정된 날짜를 명시하고 있어 최신 정보임을 강조하고 있다.

**차이점:**
1. **구체적인 내용**: Chunk 1은 공제 대상 및 그 판별 기준에 대한 상세한 규정을 다루고 있지만, Chunk 2는 주로 종합소득과세표준에 따른 구체적인 세율을 나열하고 있다.
2. **구조적 차이**: Chunk 1은 여러 조항을 포함하고 있어 더 포괄적인 내용을 담고 있는 반면, Chunk 2는 세율에 대한 표 형식으로 명확하고 간결하게 정보를 제공한다.
3. **세액 공제 조항**: Chunk 2는 배당세액공제를 포함한 세액 공제에 대한 내용을 포함하고 있어, Chunk 1과는 다른 세액 관련 규정을 다루고 있다.

### 결론
두 Chunk는 세법과 관련된 내용을 담고 있지만, 각각의 초점과 구조에서 차이를 보인다. Chunk 1은 공제 규정에 중점을 두고 있으며, Chunk 2는 구체적인 세율과 세액 공제에 대한 상세 정보를 제공한다. 이러한 차이에도 불구하고 두 텍스트는 모두 세법에 대한 이해를 돕기 위한 중요한 정보를 제공하고 있다. 전체적인 유사도를 75로 평가하며, 이는 두 문서가 주제는 비슷하지만 접근 방식과 상세 내용에서 차이를 보이기 때문임을 반영한다.
